In [197]:
import pandas as pd
import numpy as np
from random import randint
df_in = pd.read_csv('County_of_Los_Angeles_Employee_Statistics.csv')
df_in.head()

,DEPARTMENT CODE,DEPARTMENT NAME,GENDER CODE,GENDER DESCRIPTION,ETHNICITY CODE,ETHNICITY DESCRIPTION,EQUAL EMPLOYEE OPPORTUNITY JOB CATEGORY,EQUAL EMPLOYEE OPPORTUNITY JOB CATEGORY DESCRIPTION,FEDERAL EQUAL EMPLOYEE OPPORTUNITY CATEGORY CD,FEDERAL EQUAL EMPLOYEE OPPORTUNITY DESCRIPTION,EQUAL EMPLOYEE OPPORTUNITY FUNCTION CODE,EQUAL EMPLOYEE OPPORTUNITY FUNCTION DESCRIPTION,EMPLOYEE COUNT,Quarter,YEAR
0,AD,Alternate Public Defender,F,Female,1.0,WHITE,A,MANAGEMENT,A,Official & Admin,A,FINANCIAL ADMIN GENERAL CONTROL,1,1,2015
1,AD,Alternate Public Defender,F,Female,1.0,WHITE,A,MANAGEMENT,A,Official & Admin,A,FINANCIAL ADMIN GENERAL CONTROL,1,1,2016
2,AD,Alternate Public Defender,F,Female,1.0,WHITE,A,MANAGEMENT,A,Official & Admin,A,FINANCIAL ADMIN GENERAL CONTROL,1,1,2017
3,AD,Alternate Public Defender,F,Female,1.0,WHITE,A,MANAGEMENT,A,Official & Admin,A,FINANCIAL ADMIN GENERAL CONTROL,1,2,2015
4,AD,Alternate Public Defender,F,Female,1.0,WHITE,A,MANAGEMENT,A,Official & Admin,A,FINANCIAL ADMIN GENERAL CONTROL,1,2,2016


In [254]:
df = df_in.drop(['DEPARTMENT CODE', 'GENDER DESCRIPTION','EQUAL EMPLOYEE OPPORTUNITY JOB CATEGORY', 
                 'FEDERAL EQUAL EMPLOYEE OPPORTUNITY CATEGORY CD', 'EQUAL EMPLOYEE OPPORTUNITY FUNCTION CODE',
                'EMPLOYEE COUNT', 'Quarter', 'YEAR', 'DEPARTMENT NAME', 'EQUAL EMPLOYEE OPPORTUNITY JOB CATEGORY DESCRIPTION',
                'FEDERAL EQUAL EMPLOYEE OPPORTUNITY DESCRIPTION'], 1)
df.head()

,GENDER CODE,ETHNICITY CODE,ETHNICITY DESCRIPTION,EQUAL EMPLOYEE OPPORTUNITY FUNCTION DESCRIPTION
0,F,1.0,WHITE,FINANCIAL ADMIN GENERAL CONTROL
1,F,1.0,WHITE,FINANCIAL ADMIN GENERAL CONTROL
2,F,1.0,WHITE,FINANCIAL ADMIN GENERAL CONTROL
3,F,1.0,WHITE,FINANCIAL ADMIN GENERAL CONTROL
4,F,1.0,WHITE,FINANCIAL ADMIN GENERAL CONTROL


In [257]:
#deal with null and nan values (in our case, if ethnicity is mixed/not confirmed, ethnicity code is NaN)
# we have 9 eth. codes, so we change NaN to 10
df1 = df.index[df.isnull().any(axis=1)].tolist()
df.iloc[df1,1] = 10.0
len(df[df.isnull().any(axis=1)])

0

In [333]:
def unique_vals(df, col):
    """Find the unique values for a column in a dataset."""
    rows = df.iloc[:, col].values
    return set([row for row in rows])

#train dataset
train_df_in = pd.DataFrame.sample(df, 200)
train_df_in.head()

,GENDER CODE,ETHNICITY CODE,ETHNICITY DESCRIPTION,EQUAL EMPLOYEE OPPORTUNITY FUNCTION DESCRIPTION
49594,M,3.0,BLACK OR AFRICAN AMERICAN,HOSPITALS AND SANITARIUMS
15969,F,1.0,WHITE,HOSPITALS AND SANITARIUMS
12111,F,1.0,WHITE,HOSPITALS AND SANITARIUMS
37621,F,7.0,ASIAN / PACIFIC ISLANDERS,FINANCIAL ADMIN GENERAL CONTROL
17062,M,8.0,FILIPINO,HOSPITALS AND SANITARIUMS


In [334]:
classes = list(unique_vals(train_df_in, 3))
classes

['UTILITIES AND TRANSPORTATION',
 'CORRECTIONS',
 'HOSPITALS AND SANITARIUMS',
 'FIRE PROTECTION',
 'COMMUNITY DEVELOPMENT',
 'HEALTH',
 'NATURAL PARKS AND RECREATION',
 'POLICE PROTECTION',
 'OTHER (SUPPORT SERVICES, LIBRARY AND ED)',
 'FINANCIAL ADMIN GENERAL CONTROL',
 'PUBLIC WELFARE']

In [335]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [262]:
class_counts(train_df_in.values)

{'FINANCIAL ADMIN GENERAL CONTROL': 64,
 'HEALTH': 40,
 'HOSPITALS AND SANITARIUMS': 30,
 'COMMUNITY DEVELOPMENT': 9,
 'PUBLIC WELFARE': 17,
 'POLICE PROTECTION': 8,
 'NATURAL PARKS AND RECREATION': 9,
 'OTHER (SUPPORT SERVICES, LIBRARY AND ED)': 9,
 'CORRECTIONS': 9,
 'UTILITIES AND TRANSPORTATION': 4,
 'FIRE PROTECTION': 1}

In [263]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [336]:
header = df.columns.values
header

array(['GENDER CODE', 'ETHNICITY CODE', 'ETHNICITY DESCRIPTION',
       'EQUAL EMPLOYEE OPPORTUNITY FUNCTION DESCRIPTION'], dtype=object)

In [337]:
train_df = train_df_in#.drop(['EQUAL EMPLOYEE OPPORTUNITY FUNCTION DESCRIPTION'], axis=1)
train_df.head()

,GENDER CODE,ETHNICITY CODE,ETHNICITY DESCRIPTION,EQUAL EMPLOYEE OPPORTUNITY FUNCTION DESCRIPTION
49594,M,3.0,BLACK OR AFRICAN AMERICAN,HOSPITALS AND SANITARIUMS
15969,F,1.0,WHITE,HOSPITALS AND SANITARIUMS
12111,F,1.0,WHITE,HOSPITALS AND SANITARIUMS
37621,F,7.0,ASIAN / PACIFIC ISLANDERS,FINANCIAL ADMIN GENERAL CONTROL
17062,M,8.0,FILIPINO,HOSPITALS AND SANITARIUMS


In [338]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [339]:
# Let's write a question for an attribute
col = 2
uniq = list(unique_vals(train_df, col))

q = Question(col, uniq[randint(0,len(uniq)-1)])
q

Is ETHNICITY DESCRIPTION == ASIAN / PACIFIC ISLANDERS?

In [341]:
# Let's pick an example from the training set...
example = train_df.values[1]
# ... and see if it matches the question
print(train_df.values[1])
q.match(example)

['F' 1.0 'WHITE' 'HOSPITALS AND SANITARIUMS']


False

In [342]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [343]:
#######
# Demo:
# Let's partition the training data based on whether ethnicity is ASIAN / PACIFIC ISLANDERS
true_rows, false_rows = partition(train_df.values, q)
# This will contain all the 'ASIAN / PACIFIC ISLANDERS' rows.
true_rows[:5]

[array(['F', 7.0, 'ASIAN / PACIFIC ISLANDERS',
        'FINANCIAL ADMIN GENERAL CONTROL'], dtype=object),
 array(['F', 7.0, 'ASIAN / PACIFIC ISLANDERS', 'COMMUNITY DEVELOPMENT'],
       dtype=object),
 array(['F', 7.0, 'ASIAN / PACIFIC ISLANDERS',
        'FINANCIAL ADMIN GENERAL CONTROL'], dtype=object),
 array(['M', 7.0, 'ASIAN / PACIFIC ISLANDERS',
        'FINANCIAL ADMIN GENERAL CONTROL'], dtype=object),
 array(['F', 7.0, 'ASIAN / PACIFIC ISLANDERS', 'HEALTH'], dtype=object)]

In [344]:
# This will contain everything else.
false_rows[:5]

[array(['M', 3.0, 'BLACK OR AFRICAN AMERICAN', 'HOSPITALS AND SANITARIUMS'],
       dtype=object),
 array(['F', 1.0, 'WHITE', 'HOSPITALS AND SANITARIUMS'], dtype=object),
 array(['F', 1.0, 'WHITE', 'HOSPITALS AND SANITARIUMS'], dtype=object),
 array(['M', 8.0, 'FILIPINO', 'HOSPITALS AND SANITARIUMS'], dtype=object),
 array(['M', 3.0, 'BLACK OR AFRICAN AMERICAN',
        'FINANCIAL ADMIN GENERAL CONTROL'], dtype=object)]

In [345]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [346]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [347]:
# Calculate the uncertainy of our training data.
uncertainty = gini(train_df)
uncertainty

0.9998

In [348]:
# How much information do we gain by partioning on sex?
col = 0
uniq = list(unique_vals(train_df_in, col))
q1 = Question(col, uniq[randint(0,len(uniq)-1)])
q1

Is GENDER CODE == M?

In [349]:
true_rows, false_rows = partition(train_df_in.values, q1)
info_gain(true_rows, false_rows, uncertainty)

0.16750238935376544

In [350]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0
    best_question = None 
    current_uncertainty = gini(rows.values)
    n_features = len(rows.columns.values) - 1 

    for col in range(n_features):  
        values = list(unique_vals(rows, col))

        for val in values:
            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = partition(rows.values, question)
            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [351]:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(train_df_in)
best_question

Is ETHNICITY DESCRIPTION == AMERICAN INDIAN OR ALASKA NATIVE?

In [352]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows.values)

In [353]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [354]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    tr, fa = partition(rows.values, question)
    true_rows, false_rows = pd.DataFrame(np.vstack(tr)), pd.DataFrame(np.vstack(fa))
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [355]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    print (spacing + str(node.question))

    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [356]:
my_tree = build_tree(train_df)

In [357]:
print_tree(my_tree)

Is ETHNICITY DESCRIPTION == AMERICAN INDIAN OR ALASKA NATIVE?
--> True:
  Predict {'CORRECTIONS': 1}
--> False:
  Is ETHNICITY DESCRIPTION == NATIVE HAWAIIAN or OTHER PACIFIC ISLANDER?
  --> True:
    Predict {'POLICE PROTECTION': 1}
  --> False:
    Is ETHNICITY DESCRIPTION == AMERICAN INDIAN?
    --> True:
      Is GENDER CODE == M?
      --> True:
        Predict {'PUBLIC WELFARE': 1, 'NATURAL PARKS AND RECREATION': 1}
      --> False:
        Predict {'HOSPITALS AND SANITARIUMS': 1}
    --> False:
      Is ETHNICITY DESCRIPTION == ASIAN?
      --> True:
        Is GENDER CODE == M?
        --> True:
          Predict {'CORRECTIONS': 2, 'HEALTH': 1, 'FINANCIAL ADMIN GENERAL CONTROL': 3, 'FIRE PROTECTION': 1, 'PUBLIC WELFARE': 1, 'POLICE PROTECTION': 1, 'HOSPITALS AND SANITARIUMS': 1}
        --> False:
          Predict {'FINANCIAL ADMIN GENERAL CONTROL': 5, 'HOSPITALS AND SANITARIUMS': 3, 'COMMUNITY DEVELOPMENT': 1, 'CORRECTIONS': 1}
      --> False:
        Is ETHNICITY DESCRIPTIO

In [358]:
def classify(row, node):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [360]:
# The tree predicts the 25st row of our
# the higher the number, the higher the chance.
classify(train_df.values[25], my_tree)

{'OTHER (SUPPORT SERVICES, LIBRARY AND ED)': 1,
 'FINANCIAL ADMIN GENERAL CONTROL': 6,
 'FIRE PROTECTION': 1,
 'HEALTH': 5,
 'NATURAL PARKS AND RECREATION': 1,
 'UTILITIES AND TRANSPORTATION': 1,
 'HOSPITALS AND SANITARIUMS': 1,
 'POLICE PROTECTION': 1}

In [361]:
train_df.values[25]

array(['M', 5.0, 'HISPANIC OR LATINO', 'FIRE PROTECTION'], dtype=object)

In [362]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [363]:
#Print probability instead of numbers
print_leaf(classify(train_df.values[25], my_tree))

{'OTHER (SUPPORT SERVICES, LIBRARY AND ED)': '5%',
 'FINANCIAL ADMIN GENERAL CONTROL': '35%',
 'FIRE PROTECTION': '5%',
 'HEALTH': '29%',
 'NATURAL PARKS AND RECREATION': '5%',
 'UTILITIES AND TRANSPORTATION': '5%',
 'HOSPITALS AND SANITARIUMS': '5%',
 'POLICE PROTECTION': '5%'}

In [364]:
for row in train_df_in.values:
    print ("Actual: %s. \n Predicted: %s \n" %
       (row[-1], print_leaf(classify(row, my_tree))))


Actual: HOSPITALS AND SANITARIUMS. 
 Predicted: {'HOSPITALS AND SANITARIUMS': '11%', 'FINANCIAL ADMIN GENERAL CONTROL': '41%', 'NATURAL PARKS AND RECREATION': '5%', 'HEALTH': '11%', 'CORRECTIONS': '5%', 'POLICE PROTECTION': '11%', 'PUBLIC WELFARE': '5%', 'FIRE PROTECTION': '5%'} 

Actual: HOSPITALS AND SANITARIUMS. 
 Predicted: {'HOSPITALS AND SANITARIUMS': '22%', 'HEALTH': '11%', 'FINANCIAL ADMIN GENERAL CONTROL': '27%', 'FIRE PROTECTION': '5%', 'NATURAL PARKS AND RECREATION': '11%', 'CORRECTIONS': '11%', 'POLICE PROTECTION': '11%'} 

Actual: HOSPITALS AND SANITARIUMS. 
 Predicted: {'HOSPITALS AND SANITARIUMS': '22%', 'HEALTH': '11%', 'FINANCIAL ADMIN GENERAL CONTROL': '27%', 'FIRE PROTECTION': '5%', 'NATURAL PARKS AND RECREATION': '11%', 'CORRECTIONS': '11%', 'POLICE PROTECTION': '11%'} 

Actual: FINANCIAL ADMIN GENERAL CONTROL. 
 Predicted: {'FINANCIAL ADMIN GENERAL CONTROL': '42%', 'COMMUNITY DEVELOPMENT': '14%', 'HEALTH': '28%', 'NATURAL PARKS AND RECREATION': '14%'} 

Actual: HOS